In [2]:
import numpy as np
from scipy import linalg as splinalg
import matplotlib.pyplot as plt
from scipy import sparse as sp
import scipy.sparse.linalg as splin
from functools import reduce
import itertools
from scipy import linalg
from scipy.linalg import expm, logm
from scipy.special import comb
from itertools import combinations_with_replacement, product
from collections import Counter
import copy
from scipy.linalg import ishermitian
from scipy.linalg import eigh

In [3]:
Z = sp.csc_matrix([[1, 0], [0, -1]])
X = sp.csc_matrix([[0, 1], [1, 0]])
Y = sp.csc_matrix([[0, -1j], [1j, 0]])
I = sp.csc_matrix([[1, 0], [0, 1]])

params={}
params['sites'] = 4
params['I'] = sp.eye(2)
params['X'] = X
params['Y'] = Y
params['Z'] = Z

In [4]:
print(params['Y'].toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [5]:
print(Y.toarray())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [6]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
# H.toarray()

In [7]:
H = sp.csc_matrix((2**params['sites'], 2**params['sites']))
for i in range(params['sites']-1):
    H += reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(params['sites']-2-i)))) + reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(params['sites']-2-i))))

H

<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 40 stored elements and shape (16, 16)>

In [8]:
eigenvalues, eigenvectors = splin.eigsh(H, k=1, which='SA')

In [9]:
eigenvalues

array([-6.46410162])

In [10]:
eigenvectors
ground_state = eigenvectors

In [14]:
def generate_two_site_pauli_operators(N):
    two_site_ops = []
    for i in range(N - 1): 
        X_iX_i1 = reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(N - i - 2))))
        Y_iY_i1 = reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(N - i - 2))))
        Z_iZ_i1 = reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(N - i - 2))))
        two_site_ops.extend([X_iX_i1, Y_iY_i1, Z_iZ_i1])
    return two_site_ops

two_site_operators = generate_two_site_pauli_operators(params['sites'])

num_ops = len(two_site_operators)
covariance_matrix = np.zeros((num_ops, num_ops), dtype=complex)
# covariance_matrix = sp.csc_matrix((num_ops, num_ops))

# for i in range(num_ops):
#     for j in range(num_ops):
#         V_i = two_site_operators[i]
#         V_j = two_site_operators[j]
#         # Covariance matrix element: ⟨ψ| V_i V_j |ψ⟩ - ⟨ψ| V_i |ψ⟩ ⟨ψ| V_j |ψ⟩
#         term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
#         term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
#         # term1 = (ground_state.T.conj())@V_i@V_j@ground_state)
#         # term2 = ((ground_state.T.conj())@V_i@ground_state) * ((ground_state.T.conj())@(V_j@ground_state))
#         covariance_matrix[i, j] = term1 - term2

## Correlation matrix method
num_ops = len(two_site_operators)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray()
        V_j = two_site_operators[j].toarray()
        anticommutator = np.dot(V_i, V_j) + np.dot(V_j, V_i)
        term1 = np.vdot(ground_state, np.dot(anticommutator, ground_state))
        term2 = 2 * np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        correlation_matrix[i, j] = term1 - term2

eigenvalues, eigenvectors = eigh(covariance_matrix)

## Eigenvectors with zero eigenvalue (local operators that commute with the ground state)
zero_eigenvectors = eigenvectors[:, np.isclose(eigenvalues, 0, atol=1e-8)]

print("Eigenvectors with zero eigenvalue of the covariance matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors[:, i])

# # Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# # Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_corr = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors.shape[1]):
    coeff = zero_eigenvectors[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_corr += V_eff

print("\nEffective Hamiltonian in the subspace of zero eigenvectors:")
print(effective_hamiltonian_corr)

Eigenvectors with zero eigenvalue of the covariance matrix (two-site Pauli operators):
Eigenvector 1:
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 2:
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 3:
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 4:
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 5:
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 6:
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
Eigenvector 7:
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
Eigenvector 8:
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
Eigenvector 9:
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]

Effective Hamiltonian in the subspace of zero eigenvectors:
[[ 3.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]

In [18]:
def generate_two_site_pauli_operators(N):
    two_site_ops = []
    for i in range(N - 1):  # Loop over adjacent qubit pairs
        X_iX_i1 = reduce(sp.kron, (sp.eye(2**i), params['X'], params['X'], sp.eye(2**(N - i - 2))))
        Y_iY_i1 = reduce(sp.kron, (sp.eye(2**i), params['Y'], params['Y'], sp.eye(2**(N - i - 2))))
        Z_iZ_i1 = reduce(sp.kron, (sp.eye(2**i), params['Z'], params['Z'], sp.eye(2**(N - i - 2))))
        two_site_ops.extend([X_iX_i1, Y_iY_i1, Z_iZ_i1])
    return two_site_ops

two_site_operators = generate_two_site_pauli_operators(params['sites'])

# Construct the correlation matrix
num_ops = len(two_site_operators)
correlation_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray()
        V_j = two_site_operators[j].toarray()
        # Correlation matrix element: ⟨ψ| {V_i, V_j} |ψ⟩ - 2 ⟨ψ| V_i |ψ⟩ ⟨ψ| V_j |ψ⟩
        anticommutator = np.dot(V_i, V_j) + np.dot(V_j, V_i)
        term1 = np.vdot(ground_state, np.dot(anticommutator, ground_state))
        term2 = 2 * np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        correlation_matrix[i, j] = term1 - term2

# Diagonalize the correlation matrix
eigenvalues_corr, eigenvectors_corr = eigh(correlation_matrix)

# Identify zero eigenvectors (local operators that commute with the ground state)
zero_eigenvectors_corr = eigenvectors_corr[:, np.isclose(eigenvalues_corr, 0, atol=1e-8)]

# Print the zero eigenvectors (local operators with |ψ⟩ as an eigenstate)
print("Zero eigenvectors of the correlation matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors_corr.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors_corr[:, i])

# Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_corr = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors_corr.shape[1]):
    coeff = zero_eigenvectors_corr[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_corr += V_eff
import numpy as np
import scipy.sparse as sp
from scipy.linalg import eigh
from functools import reduce
import matplotlib.pyplot as plt

# Parameters
L = 10  # System size
h = 1.0  # Magnetic field
D = 0.1  # Anisotropy
J = 0.1  # Coupling strength

# Define spin-1 operators
Sx = sp.csc_matrix([[0, 1, 0], [1, 0, 1], [0, 1, 0]]) / np.sqrt(2)
Sy = sp.csc_matrix([[0, -1j, 0], [1j, 0, -1j], [0, 1j, 0]]) / np.sqrt(2)
Sz = sp.csc_matrix([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
I = sp.eye(3)

# Function to compute tensor product of a list of operators
def tensor(operators):
    return reduce(sp.kron, operators)

# Construct the 1D spin-1 XY model Hamiltonian
def spin1_xy_hamiltonian(L, h, D, J):
    H = sp.csc_matrix((3**L, 3**L))
    for i in range(L - 1):  # Nearest-neighbor interactions
        H += J * reduce(sp.kron, [sp.eye(3**i), Sx, Sx, sp.eye(3**(L - i - 2))])
        H += J * reduce(sp.kron, [sp.eye(3**i), Sy, Sy, sp.eye(3**(L - i - 2))])
    for i in range(L):  # On-site terms
        H += h * reduce(sp.kron, [sp.eye(3**i), Sz, sp.eye(3**(L - i - 1))])
        H += D * reduce(sp.kron, [sp.eye(3**i), Sz @ Sz, sp.eye(3**(L - i - 1))])
    return H

# Compute the Hamiltonian
H = spin1_xy_hamiltonian(L, h, D, J)

# Diagonalize the Hamiltonian
eigenvalues, eigenvectors = eigh(H.toarray())

# Function to compute bipartite entanglement entropy
def entanglement_entropy(state, L, partition):
    """Compute the entanglement entropy for a bipartition of the system."""
    # Reshape the state into a matrix for the bipartition
    state_matrix = state.reshape((3**partition, 3**(L - partition)))
    # Perform singular value decomposition (SVD)
    U, s, Vh = np.linalg.svd(state_matrix, full_matrices=False)
    # Compute the entanglement entropy
    s_squared = s**2
    s_squared = s_squared[s_squared > 1e-12]  # Remove numerical noise
    return -np.sum(s_squared * np.log(s_squared))

# Compute entanglement entropy for each eigenstate
partition = L // 2  # Bipartition the system into two equal halves
entropies = [entanglement_entropy(eigenvectors[:, i], L, partition) for i in range(3**L)]

# Plot EE vs. Energy
plt.figure(figsize=(10, 6))
plt.scatter(eigenvalues, entropies, s=10, label="Eigenstates")
# Highlight QMBS states (example: states with EE < 1)
qmbs_indices = np.where(np.array(entropies) < 1)[0]
plt.scatter(eigenvalues[qmbs_indices], np.array(entropies)[qmbs_indices], color="red", s=50, label="QMBS States")
plt.xlabel("Eigenstate Energy $E$")
plt.ylabel("Bipartite Entanglement Entropy $S_A$")
plt.title(f"Entanglement Entropy vs. Energy (L = {L}, h = {h}, D = {D}, J = {J})")
plt.legend()
plt.grid(True)

# Inset: Logarithmic scaling of EE for the QMBS state
plt.axes([0.6, 0.6, 0.25, 0.25])
system_sizes = [6, 8, 10, 12]  # Example system sizes
qmbs_entropies = []  # Entanglement entropies for the QMBS state
for size in system_sizes:
    H_size = spin1_xy_hamiltonian(size, h, D, J)
    eigenvalues_size, eigenvectors_size = eigh(H_size.toarray())
    qmbs_state = eigenvectors_size[:, 0]  # Example: ground state as QMBS state
    qmbs_entropies.append(entanglement_entropy(qmbs_state, size, size // 2))
plt.plot(system_sizes, qmbs_entropies, marker="o", linestyle="-", color="red")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("System Size $L$")
plt.ylabel("Entanglement Entropy $S_A$")
plt.title("Logarithmic Scaling of EE")

plt.show()
# Print the effective Hamiltonian
print("\nEffective Hamiltonian in the subspace of zero eigenvectors (correlation matrix method):")
print(effective_hamiltonian_corr)

# Now compare with the covariance matrix method
# Construct the covariance matrix
covariance_matrix = np.zeros((num_ops, num_ops), dtype=complex)

for i in range(num_ops):
    for j in range(num_ops):
        V_i = two_site_operators[i].toarray()
        V_j = two_site_operators[j].toarray()
        term1 = np.vdot(ground_state, np.dot(V_i, np.dot(V_j, ground_state)))
        term2 = np.vdot(ground_state, np.dot(V_i, ground_state)) * np.vdot(ground_state, np.dot(V_j, ground_state))
        covariance_matrix[i, j] = term1 - term2

# Diagonalize the covariance matrix
eigenvalues_cov, eigenvectors_cov = eigh(covariance_matrix)

# Identify zero eigenvectors (local operators that commute with the ground state)
zero_eigenvectors_cov = eigenvectors_cov[:, np.isclose(eigenvalues_cov, 0, atol=1e-8)]

# Print the zero eigenvectors (local operators with |ψ⟩ as an eigenstate)
print("\nZero eigenvectors of the covariance matrix (two-site Pauli operators):")
for i in range(zero_eigenvectors_cov.shape[1]):
    print(f"Eigenvector {i + 1}:")
    print(zero_eigenvectors_cov[:, i])

# Reconstruct the effective Hamiltonian in the subspace of zero eigenvectors
# Use the zero eigenvectors to construct an effective Hamiltonian
effective_hamiltonian_cov = np.zeros((2**params['sites'], 2**params['sites']), dtype=complex)
for i in range(zero_eigenvectors_cov.shape[1]):
    coeff = zero_eigenvectors_cov[:, i]
    V_eff = sum(coeff[j] * two_site_operators[j].toarray() for j in range(num_ops))
    effective_hamiltonian_cov += V_eff

# Print the effective Hamiltonian
print("\nEffective Hamiltonian in the subspace of zero eigenvectors (covariance matrix method):")
print(effective_hamiltonian_cov)

# Compare the results
print("\nComparison of effective Hamiltonians:")
print("Correlation matrix method vs. Covariance matrix method")
print(np.allclose(effective_hamiltonian_corr, effective_hamiltonian_cov, atol=1e-8))

Zero eigenvectors of the correlation matrix (two-site Pauli operators):
Eigenvector 1:
[ 0.6488746 +0.j         -0.41811388-0.03001023j -0.20950543+0.04498597j
 -0.08033743+0.01355825j  0.02025443+0.03276306j  0.01293109-0.01063006j
 -0.50376372+0.01925578j  0.28840266-0.00320251j  0.09980195+0.04035349j]
Eigenvector 2:
[ 0.        +0.j         -0.33974788-0.08751565j  0.78233509+0.24068775j
  0.03315343+0.04441801j  0.11714525+0.03515757j -0.14437055-0.06973657j
 -0.08122611-0.1058327j   0.02905183+0.00698296j -0.37855667-0.03464433j]
Eigenvector 3:
[ 0.        +0.j          0.16332259-0.04297011j -0.1374999 -0.06908393j
 -0.04760704-0.05605683j -0.26490613+0.03674105j  0.29001902+0.01268352j
  0.09458369+0.14268871j  0.52493326-0.06786972j -0.69032794+0.02397051j]
Eigenvector 4:
[-5.62613415e-01-0.j         -2.34335564e-01-0.06609344j
 -2.64212431e-04+0.0327973j   2.65592571e-01-0.01034242j
 -1.30788628e-01+0.02355626j -1.29557930e-01-0.03344978j
 -1.54724183e-01-0.00366313j  5.99931

MemoryError: Unable to allocate 52.0 GiB for an array with shape (59049, 59049) and data type complex128